# Computing the Jacobian matrix of a neural net

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tqdm import tqdm

Say we have a simple neural network, with two affine layers followed by a ReLU and softmax non-linearities, respectively:

In [2]:
N = 500  # Input size
H = 100  # Hidden layer size
O = 10   # Output size

w1 = np.random.randn(N, H)
b1 = np.random.randn(H)

w2 = np.random.randn(H, O)
b2 = np.random.randn(O)

In [3]:
""" Numpy implementation
"""

def ffpass_np(x):
    a1 = np.dot(x, w1) + b1   # affine
    r = np.maximum(0, a1)    # ReLU
    a2 = np.dot(r, w2) + b2  # affine
    
    exps = np.exp(a2 - np.max(a2))  # softmax
    out = exps / exps.sum()
    return out

Here the outputs would typically denote class propabilities; in which case some classification loss would be used for training.
We do not focus on training here, but just on computing the Jacobian matrix of an existing neural network.

Let's also write a Keras implementation:

In [4]:
""" Keras implementation
"""
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

model = tf.keras.Sequential()
model.add(Dense(H, activation='relu', use_bias=True, input_dim=N))
model.add(Dense(O, activation='softmax', use_bias=True, input_dim=O))
model.get_layer(index=0).set_weights([w1, b1])
model.get_layer(index=1).set_weights([w2, b2])
    
def ffpass_tf(x):
    xr = x.reshape((1, x.size))
    return model.predict(xr)[0]

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Do our two implementations agree?

In [5]:
x0 = np.random.random((N,))
# x0 /= sum(x0)

out_np = ffpass_np(x0)
out_keras = ffpass_tf(x0)

np.allclose(out_np, out_keras, 1e-4)

True

## Jacobian matrix computation
### Option 1: Tensorflow
Unfortunately, tensorflow only provides computation of the gradient (with respect to a scalar output) -- see: https://github.com/tensorflow/tensorflow/issues/675.

However, we can still iterate over each output, compute its gradient vector, and then group those vectors into a Jacobian matrix.

In [6]:
def jacobian_tensorflow(x, verbose=False):    
    jacobian_matrix = []
    it = tqdm(range(O)) if verbose else range(O)
    for o in it:
        grad_func = tf.gradients(model.output[:, o], model.input)
        gradients = sess.run(grad_func, feed_dict={model.input: x.reshape((1, x.size))})
        jacobian_matrix.append(gradients[0][0,:])
        
    return np.array(jacobian_matrix)

In [7]:
import time

tic = time.time()
jacobian_tf = jacobian_tensorflow(x0, verbose=False)
# %timeit jacobian_tf = jacobian_tensorflow(x0, verbose=False)
tac = time.time()

print('It took %.3f s. to compute the Jacobian matrix using Tensorflow' % (tac-tic))

It took 0.658 s. to compute the Jacobian matrix using Tensorflow


Is the Jacobian computation correct?

In [10]:
def is_jacobian_correct(jacobian_fn, ffpass_fn):
    """ Numerical check of the Jacobian
    """
    x = np.random.random((N,))
    # x /= sum(x)
    epsilon = 1e-5

    """ Check a few columns at random
    """
    for idx in np.random.choice(N, 5, replace=False):
    # for idx in range(5):
        x2 = x.copy()
        x2[idx] += epsilon

        num_jacobian = (ffpass_fn(x2) - ffpass_fn(x)) / epsilon
        computed_jacobian = jacobian_fn(x)
        
        if not all(abs(computed_jacobian[:, idx] - num_jacobian) < 1e-3):
            
            print('Found a mismatch.')
            print('Numerical: {}'.format(num_jacobian[:10]))
            print('Computed: {}'.format(computed_jacobian[:10, idx]))
            
            return False

    return True

In [11]:
is_jacobian_correct(jacobian_tensorflow, ffpass_tf)

True

## Option 2: Autograd
Autograd provides out-of-the-box automatic differentiation for Numpy-based functions.

We have to start by re-defining our feedforward pass using the autograd's encapsulated Numpy.

In [12]:
import autograd.numpy as anp

def ffpass_anp(x):
    a1 = anp.dot(x, w1) + b1   # affine
    a1 = anp.maximum(0, a1)    # ReLU
    a2 = anp.dot(a1, w2) + b2  # affine
    
    exps = anp.exp(a2 - anp.max(a2))  # softmax
    out = exps / exps.sum()
    return out

In [13]:
# Check the output
out_anp = ffpass_anp(x0)
out_keras = ffpass_tf(x0)

np.allclose(out_anp, out_keras, 1e-4)

True

In [14]:
# Compute Jacobian
from autograd import jacobian

def jacobian_autograd(x):
    return jacobian(ffpass_anp)(x)

In [15]:
# Is it correct?
is_jacobian_correct(jacobian_autograd, ffpass_np)

True

In [16]:
%timeit jacobian_autograd(x0)

3.69 ms ± 135 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Do our two Jacobian matrices agree?

In [23]:
jacobian_tf = jacobian_tensorflow(x0, verbose=False)
jacobian_a = jacobian_autograd(x0)

np.allclose(jacobian_tf, jacobian_a, 1e-2)

True

## Option 3: write your backprop with Numpy
Let's try to just re-implement our own backpropagation for this neural net. Here, I re-use the same kind of layer formalism as you can find in the http://cs231n.stanford.edu/ class.

In [17]:
def affine_forward(x, w, b):
    """
    Forward pass of an affine layer
    :param x: input of dimension (D, )
    :param w: weights matrix of dimension (D, M)
    :param b: biais vector of dimension (M, )
    :return output of dimension (M, ), and cache needed for backprop
    """
    out = np.dot(x, w) + b
    cache = (x, w)
    return out, cache


def affine_backward(dout, cache):
    """
    Backward pass for an affine layer.
    :param dout: Upstream Jacobian, of shape (O, M)
    :param cache: Tuple of:
      - x: Input data, of shape (D, )
      - w: Weights, of shape (D, M)
    :return the jacobian matrix containing derivatives of the O neural network outputs with respect to
            this layer's inputs, evaluated at x, of shape (O, D)
    """
    x, w = cache
    dx = np.dot(dout, w.T)
    return dx


def relu_forward(x):
    """ Forward ReLU
    """
    out = np.maximum(np.zeros(x.shape), x)
    cache = x
    return out, cache


def relu_backward(dout, cache):
    """
    Backward pass of ReLU
    :param dout: Upstream Jacobian
    :param cache: the cached input for this layer
    :return: the jacobian matrix containing derivatives of the O neural network outputs with respect to
             this layer's inputs, evaluated at x.
    """
    x = cache
    dx = dout * np.where(x > 0, np.ones(x.shape), np.zeros(x.shape))
    return dx

def softmax_forward(x):
    """ Forward softmax
    """
    exps = np.exp(x - np.max(x))
    s = exps / exps.sum()
    return s, s
    
def softmax_backward(dout, cache):
    """
    Backward pass for softmax
    :param dout: Upstream Jacobian
    :param cache: contains the cache (in this case the output) for this layer
    """
    s = cache
    ds = np.diag(s) - np.outer(s, s.T)
    dx = np.dot(dout, ds)
    return dx

In [18]:
def forward_backward(x):
    layer_to_cache = dict()  # for each layer, we store the cache needed for backward pass

    # Forward pass:
    a1, cache_a1 = affine_forward(x, w1, b1)
    r1, cache_r1 = relu_forward(a1)
    a2, cache_a2 = affine_forward(r1, w2, b2)
    out, cache_out = softmax_forward(a2)

    # backward pass
    dout = np.diag(np.ones(out.size, ))  # the derivatives of each output w.r.t. each output.
    dout = softmax_backward(dout, cache_out)
    dout = affine_backward(dout, cache_a2)
    dout = relu_backward(dout, cache_r1)
    dx = affine_backward(dout, cache_a1)
    
    return out, dx

In [19]:
# Check the output
out_fb = forward_backward(x0)[0]
out_keras = ffpass_tf(x0)

np.allclose(out_fb, out_keras, 1e-4)

True

In [20]:
# Check Jacobian
is_jacobian_correct(lambda x: forward_backward(x)[1], ffpass_tf)

True

In [22]:
%timeit forward_backward(x0)

115 µs ± 2.38 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Conclusions
For $N=500, H=100$ and $O=10$, I get about 650 ms for Tensorflow (granted, probably mine is not the best possible implementation), 3.5 ms with autograd, and 110 µs with Numpy.

So for this problem, a home made Numpy implementation can be about 30 times faster than autograd, which is itself about 200 times faster than Tensorflow.